In [1]:
%%file dataset.py
# Generate training dataset and test dataset.
from constant import *
import os
import numpy as np
from PIL import Image, ImageDraw, ImageColor
import xml.etree.ElementTree as ET
import copy
import random
import time
import argparse
import math
import act_max as act
import scipy.misc, scipy.io
#import caffe

#### Parameters. ####
parser = argparse.ArgumentParser()
parser.add_argument('--dataset_index', required=True)
parser.add_argument('--type_str', required=True)
#parser.add_argument('--gpu', required=False)
args = parser.parse_args()

#type_str = 'crop_img' # {1k_}[crop | nocrop | crop_obj | nocrop_obj | aperture | crop_img ]
type_str = args.type_str

#dataset = [(0.0, 0),  (0.1, 10), (0.2, 10), (0.3, 10), (0.4, 10),\
#           (0.5, 10), (0.6, 10), (0.7, 10), (0.8, 10), (0.9, 10), (1.0, 1)]
dataset = [(0.0, 0),  (0.1, 1), (0.2, 1), (0.3, 1), (0.4, 1),\
           (0.5, 1), (0.6, 1), (0.7, 1), (0.8, 1), (0.9, 1), (1.0, 1)]
dataset_index = int(args.dataset_index)
#dataset_index = 5
print 'Processsing dataset {}, type_str {}'.format(dataset[dataset_index], type_str)
dataset = [dataset[dataset_index]]

'''
if args.gpu != None:
    gpu = int(args.gpu)
    caffe.set_device(gpu)
    caffe.set_mode_gpu()
'''
#caffe.set_mode_gpu()

# divide to training dataset and test dataset
training_dataset_size = 300
validation_dataset_size = 100
test_dataset_size = 100

#####################

mean_color = (123, 117, 104)

# Load labels.
imagenet_labels_filename = imagenet_root + 'ilsvrc12/synset_words.txt'
label_to_wnid = np.loadtxt(imagenet_labels_filename, str, delimiter='\t')
wnid_to_label = {}
for label in range(len(label_to_wnid)):
    wnid = label_to_wnid[label].split(' ')[0]
    wnid_to_label[wnid] = label

train_folders = []
val_folders = []
test_folders = []

train_files = []
val_files = []
test_files = []

for (slider_size, slider_num) in dataset:
    percent = str(int(100 * slider_size))
    f = open('{}dataset/train_{}_{}.txt'.format(imagenet_root, type_str, percent), 'w')
    train_files.append(f)
    
    folder = '{}dataset/train_{}_{}/'.format(imagenet_root, type_str, percent)
    if not os.path.exists(folder):
        os.makedirs(folder)
    train_folders.append(folder)

for (slider_size, slider_num) in dataset:
    percent = str(int(100 * slider_size))
    f = open('{}dataset/val_{}_{}.txt'.format(imagenet_root, type_str, percent), 'w')
    val_files.append(f)

    folder = '{}dataset/val_{}_{}/'.format(imagenet_root, type_str, percent)
    if not os.path.exists(folder):
        os.makedirs(folder)
    val_folders.append(folder)

for (slider_size, slider_num) in dataset:
    percent = str(int(100 * slider_size))
    f = open('{}dataset/test_{}_{}.txt'.format(imagenet_root, type_str, percent), 'w')
    test_files.append(f)

    folder = '{}dataset/test_{}_{}/'.format(imagenet_root, type_str, percent)
    if not os.path.exists(folder):
        os.makedirs(folder)
    test_folders.append(folder)
    
# Initialization for 'obj'.
if 'obj' in type_str:
    obj_images = []
    for i, image_name in enumerate(os.listdir(shapenet_root + 'object_crop/')):
        img_temp = Image.open(shapenet_root + 'object_crop/' + image_name)
        img = img_temp.copy()
        img_temp.close()
        #img = img.convert("RGBA")
        obj_images.append(img)
    print "{} object occluders loaded.".format(len(obj_images))
    
'''
# On-the-fly generation.
# Initialization for 'imagination'.
if type_str == 'crop_img':
    act_args = lambda: None
    act_args.xy = 0
    act_args.n_iters = 200
    act_args.L2 = 0.99
    act_args.start_lr = 8.0
    act_args.end_lr = 1e-10
    act_args.seed = 0
    act_args.opt_layer = "fc6"
    act_args.act_layer = "fc8_occlusion"
    act_args.init_file = "None"
    act_args.clip = 0
    act_args.bound = synthesizing_root + "act_range/3x/fc6.txt"
    act_args.debug = 0
    act_args.output_dir = synthesizing_root + "output/baseline"

    act_args.net_weights = result_root + "model/finetune_alexnet_crop_0/finetune_alexnet_crop_0.caffemodel"
    act_args.net_definition = result_root + "model/finetune_alexnet_crop_0/deploy.prototxt"
    act_args.generator_weights = synthesizing_root + "nets/upconv/fc6/generator.caffemodel"
    act_args.generator_definition = synthesizing_root + "nets/upconv/fc6/generator.prototxt"
    act_args.encoder_weights = synthesizing_root + "nets/caffenet/bvlc_reference_caffenet.caffemodel"
    act_args.encoder_definition = synthesizing_root + "nets/caffenet/caffenet.prototxt"

    params = [
        {
            'layer': act_args.act_layer,
            'iter_n': act_args.n_iters,
            'L2': act_args.L2,
            'start_step_size': act_args.start_lr,
            'end_step_size': act_args.end_lr
        }
    ]

    # Networks.
    generator = caffe.Net(act_args.generator_definition, act_args.generator_weights, caffe.TEST)
    net = caffe.Classifier(act_args.net_definition, act_args.net_weights,
            mean = np.float32([104.0, 117.0, 123.0]), # ImageNet mean
            channel_swap = (2,1,0)) # the reference model has channels in BGR order instead of RGB

    # input / output layers in generator
    gen_in_layer = "feat"
    gen_out_layer = "deconv0"

    # shape of the code being optimized
    shape = generator.blobs[gen_in_layer].data.shape

    # Fix the seed
    np.random.seed(act_args.seed)
'''

# Initialization for 'crop_img'.
if type_str == 'crop_img':
    img_images = []
    for class_id in range(100):
        for num in range(20):
            filename = '{}_{}.jpg'.format(class_id, num)
            img_temp = Image.open(imagenet_root + 'synthesized/' + filename)
            img = img_temp.copy()
            img_temp.close()
            img_images.append(img)
    print "{} synthesized images loaded.".format(len(img_images))
    


# occluder size = slider_size * slider_size
# occluder num = slider_num * slider_num
# path = 'imagenet_root/dataset/train_0/name'
# {wnid_imgid}_{crop/nocrop}_{rect_i(ifcrop)}_{slider_size}_{i}_{j}
def generate_datum(img_orig, path, f, class_id, rects, slider_size, slider_num):
    if type_str == 'crop' or type_str == '1k_crop':
        if slider_size == 0:
            for rect_i, rect in enumerate(rects):
                img = img_orig.copy()
                img = img.crop(rect)
                datum_path = '{}_{}_{}_0_0.jpeg'.format(path, type_str, rect_i)
                img.save(datum_path)
                f.write('{} {}\n'.format(datum_path, str(class_id)))
        else:
            for rect_i, rect in enumerate(rects):
                for i in range(slider_num):
                    img = img_orig.copy()
                    d = ImageDraw.Draw(img)
                    slider_width = int((rect[2] - rect[0]) * math.sqrt(slider_size))
                    slider_height = int((rect[3] - rect[1]) * math.sqrt(slider_size))
                    subrect = [0, 0, 0, 0]
                    subrect[0] = random.randint(rect[0], rect[2] - slider_width)
                    subrect[1] = random.randint(rect[1], rect[3] - slider_height)
                    subrect[2] = subrect[0] + slider_width
                    subrect[3] = subrect[1] + slider_height
                    d.rectangle(subrect, fill=mean_color, outline=None)
                    img = img.crop(rect)
                    datum_path = '{}_{}_{}_{}_{}.jpeg'.format(path, type_str, rect_i, str(int(100 * slider_size)), i)
                    img.save(datum_path)
                    f.write('{} {}\n'.format(datum_path, str(class_id)))
    if type_str == 'nocrop' or type_str == '1k_nocrop':
        if slider_size == 0:
            datum_path = '{}_{}_0_0_0.jpeg'.format(path, type_str)
            img_orig.save(datum_path)
            f.write('{} {}\n'.format(datum_path, str(class_id)))
        else:
            for i in range(slider_num):
                img = img_orig.copy()
                d = ImageDraw.Draw(img)
                for rect in rects:
                    slider_width = int((rect[2] - rect[0]) * math.sqrt(slider_size))
                    slider_height = int((rect[3] - rect[1]) * math.sqrt(slider_size))
                    subrect = [0, 0, 0, 0]
                    subrect[0] = random.randint(rect[0], rect[2] - slider_width)
                    subrect[1] = random.randint(rect[1], rect[3] - slider_height)
                    subrect[2] = subrect[0] + slider_width
                    subrect[3] = subrect[1] + slider_height
                    d.rectangle(subrect, fill=mean_color, outline=None)
                datum_path = '{}_{}_{}_{}_{}.jpeg'.format(path, type_str, 0, str(int(100 * slider_size)), i)
                img.save(datum_path)
                f.write('{} {}\n'.format(datum_path, str(class_id)))   
                
                
    if type_str == 'aperture' or type_str == '1k_aperture':
        if slider_size == 0: # All gray.
            for rect_i, rect in enumerate(rects):
                img = img_orig.copy()
                d = ImageDraw.Draw(img)
                d.rectangle(rect, fill=mean_color, outline=None)
                img = img.crop(rect)
                datum_path = '{}_{}_{}_{}_0.jpeg'.format(path, type_str, rect_i, str(int(100 * slider_size)))
                img.save(datum_path)
                f.write('{} {}\n'.format(datum_path, str(class_id)))
        elif slider_size == 1: # All visible. 
            for rect_i, rect in enumerate(rects):
                img = img_orig.copy()
                img = img.crop(rect)
                datum_path = '{}_{}_{}_{}_0.jpeg'.format(path, type_str, rect_i, str(int(100 * slider_size)))
                img.save(datum_path)
                f.write('{} {}\n'.format(datum_path, str(class_id)))
        else:
            for rect_i, rect in enumerate(rects):
                for i in range(slider_num):
                    img = img_orig.copy()
                    d = ImageDraw.Draw(img)
                    
                    slider_width = int((rect[2] - rect[0]) * math.sqrt(slider_size))
                    slider_height = int((rect[3] - rect[1]) * math.sqrt(slider_size))
                    
                    subrect = [0, 0, 0, 0]
                    subrect[0] = random.randint(rect[0], rect[2] - slider_width)
                    subrect[1] = random.randint(rect[1], rect[3] - slider_height)
                    subrect[2] = subrect[0] + slider_width
                    subrect[3] = subrect[1] + slider_height

                    d.rectangle([0, 0, img.size[0], subrect[1]], fill=mean_color, outline=None)
                    d.rectangle([0, 0, subrect[0], img.size[1]], fill=mean_color, outline=None)
                    d.rectangle([subrect[2], 0, img.size[0], img.size[1]], fill=mean_color, outline=None)
                    d.rectangle([0, subrect[3], img.size[0], img.size[1]], fill=mean_color, outline=None)
                    img = img.crop(rect)
                    datum_path = '{}_{}_{}_{}_{}.jpeg'.format(path, type_str, rect_i, str(int(100 * slider_size)), i)
                    img.save(datum_path)
                    f.write('{} {}\n'.format(datum_path, str(class_id)))
    
    if type_str == 'crop_obj' or type_str == '1k_crop_obj':
        if slider_size == 0:
            for rect_i, rect in enumerate(rects):
                img = img_orig.copy()
                img = img.crop(rect)
                datum_path = '{}_{}_{}_{}_0.jpeg'.format(path, type_str, rect_i, int(100 * slider_size))
                img.save(datum_path)
                f.write('{} {}\n'.format(datum_path, str(class_id)))
        else:
            for rect_i, rect in enumerate(rects):
                for num in range(slider_num):
                    img = img_orig.copy()
                    random_obj = obj_images[random.randint(0, len(obj_images) - 1)].copy()
                    obj_width = random_obj.size[0]
                    obj_height = random_obj.size[1]
                    obj_ratio = float(obj_width) / obj_height
                    
                    width = rect[2] - rect[0]
                    height = rect[3] - rect[1]
                    ratio = float(width) / height
                    
                    max_occlusion = 0.0
                    if ratio >= obj_ratio:
                        max_occlusion = obj_width * height / float(width * obj_height)
                    else:
                        max_occlusion = width * obj_height / float(height * obj_width)
                    
                    if max_occlusion >= slider_size:
                        # Do not need to stretch and change obj_ratio.
                        new_width = (slider_size * width * height * obj_width / float(obj_height)) ** 0.5
                        new_height = (slider_size * width * height * obj_height / float(obj_width)) ** 0.5
                    else:
                        # Need to stretch and change obj_ratio.
                        if ratio >= obj_ratio:
                            new_width = width * slider_size
                            new_height = height
                        else:
                            new_width = width
                            new_height = height * slider_size
                    new_width = int(new_width)
                    new_height = int(new_height)
                    if new_width == 0:
                        new_width = 1
                    if new_height == 0:
                        new_height = 1
                        
                    random_obj = random_obj.resize((new_width, new_height), Image.ANTIALIAS)
                    
                    top_left = (random.randint(rect[0], rect[2] - random_obj.size[0]),\
                                random.randint(rect[1], rect[3] - random_obj.size[1]))
                    
                    img.paste(random_obj, top_left, random_obj)
                    img = img.crop(rect)
                    datum_path = '{}_{}_{}_{}_{}.jpeg'.format(path, type_str, rect_i, int(100 * slider_size), num)
                    img.save(datum_path)
                    f.write('{} {}\n'.format(datum_path, str(class_id)))
                    
    if type_str == 'nocrop_obj' or type_str == '1k_nocrop_obj':
        if slider_size == 0:
            datum_path = '{}_{}_{}_{}_0.jpeg'.format(path, type_str, 0, int(100 * slider_size))
            img_orig.save(datum_path)
            f.write('{} {}\n'.format(datum_path, str(class_id)))
        else:
            for num in range(slider_num):
                img = img_orig.copy()
                for rect_i, rect in enumerate(rects):
                    random_obj = obj_images[random.randint(0, len(obj_images) - 1)].copy()
                    
                    obj_width = random_obj.size[0]
                    obj_height = random_obj.size[1]
                    obj_ratio = float(obj_width) / obj_height
                    
                    width = rect[2] - rect[0]
                    height = rect[3] - rect[1]
                    ratio = float(width) / height
                    
                    max_occlusion = 0.0
                    if ratio >= obj_ratio:
                        max_occlusion = obj_width * height / float(width * obj_height)
                    else:
                        max_occlusion = width * obj_height / float(height * obj_width)
                    
                    if max_occlusion >= slider_size:
                        # Do not need to stretch and change obj_ratio.
                        new_width = (slider_size * width * height * obj_width / float(obj_height)) ** 0.5
                        new_height = (slider_size * width * height * obj_height / float(obj_width)) ** 0.5
                    else:
                        # Need to stretch and change obj_ratio.
                        if ratio >= obj_ratio:
                            new_width = width * slider_size
                            new_height = height
                        else:
                            new_width = width
                            new_height = height * slider_size
                    new_width = int(new_width)
                    new_height = int(new_height)
                    if new_width == 0:
                        new_width = 1
                    if new_height == 0:
                        new_height = 1
                        
                    random_obj = random_obj.resize((new_width, new_height), Image.ANTIALIAS)
                    
                    top_left = (random.randint(rect[0], rect[2] - random_obj.size[0]),\
                                random.randint(rect[1], rect[3] - random_obj.size[1]))
                    
                    img.paste(random_obj, top_left, random_obj)
                    
                datum_path = '{}_{}_{}_{}_{}.jpeg'.format(path, type_str, rect_i, int(100 * slider_size), num)
                img.save(datum_path)
                f.write('{} {}\n'.format(datum_path, str(class_id))) 
    
    if type_str == "crop_img":
        if slider_size == 0:
            for rect_i, rect in enumerate(rects):
                img = img_orig.copy()
                img = img.crop(rect)
                datum_path = '{}_{}_{}_0_0.jpeg'.format(path, type_str, rect_i)
                img.save(datum_path)
                f.write('{} {}\n'.format(datum_path, str(class_id)))
        else:
            for rect_i, rect in enumerate(rects):
                for i in range(slider_num):
                    img = img_orig.copy()
                    d = ImageDraw.Draw(img)
                    slider_width = int((rect[2] - rect[0]) * math.sqrt(slider_size))
                    slider_height = int((rect[3] - rect[1]) * math.sqrt(slider_size))
                    subrect = [0, 0, 0, 0]
                    subrect[0] = random.randint(rect[0], rect[2] - slider_width)
                    subrect[1] = random.randint(rect[1], rect[3] - slider_height)
                    subrect[2] = subrect[0] + slider_width
                    subrect[3] = subrect[1] + slider_height
                    img_image = img_images[random.randint(0, 2000 - 1)]
                    img.paste(img_image.resize((slider_width, slider_height)), (subrect[0], subrect[1]))
                    img = img.crop(rect)
                    datum_path = '{}_{}_{}_{}_{}.jpeg'.format(path, type_str, rect_i, str(int(100 * slider_size)), i)
                    img.save(datum_path)
                    f.write('{} {}\n'.format(datum_path, str(class_id)))
            
    
    '''
    # On-the-fly generation.
    if type_str == "crop_img":
        if slider_size == 0:
            for rect_i, rect in enumerate(rects):
                img = img_orig.copy()
                img = img.crop(rect)
                datum_path = '{}_{}_{}_0_0.jpeg'.format(path, type_str, rect_i)
                img.save(datum_path)
                f.write('{} {}\n'.format(datum_path, str(class_id)))
        else:
            for rect_i, rect in enumerate(rects):
                for i in range(slider_num):
                    act_args.unit = random.randint(0, 99)
                    img = img_orig.copy()
                    d = ImageDraw.Draw(img)
                    slider_width = int((rect[2] - rect[0]) * math.sqrt(slider_size))
                    slider_height = int((rect[3] - rect[1]) * math.sqrt(slider_size))
                    subrect = [0, 0, 0, 0]
                    subrect[0] = random.randint(rect[0], rect[2] - slider_width)
                    subrect[1] = random.randint(rect[1], rect[3] - slider_height)
                    subrect[2] = subrect[0] + slider_width
                    subrect[3] = subrect[1] + slider_height
                    
                    act_args.init_file = '{}_{}_{}_{}_{}_init.jpeg'.format(path, type_str, rect_i, str(int(100 * slider_size)), i)
                    img.crop(subrect).save(act_args.init_file)
                    start_code, start_image = act.get_code(act_args.init_file, act_args.opt_layer)
                    os.remove(act_args.init_file)
                    
                    #start_code, start_image = act.get_code2(np.array(img.crop(subrect)), act_args.opt_layer)
                    
                    # Load the activation range
                    upper_bound = lower_bound = None
                    # Set up clipping bounds
                    if act_args.bound != "":
                        n_units = shape[1]
                        upper_bound = np.loadtxt(act_args.bound, delimiter=' ', usecols=np.arange(0, n_units), unpack=True)
                        upper_bound = upper_bound.reshape(start_code.shape)
                        # Lower bound of 0 due to ReLU
                        lower_bound = np.zeros(start_code.shape)
                    # Optimize a code via gradient ascent
                    output_image = act.activation_maximization(net, generator, gen_in_layer, gen_out_layer, start_code, params, 
                            clip=act_args.clip, unit=act_args.unit, xy=act_args.xy, debug=act_args.debug,
                            upper_bound=upper_bound, lower_bound=lower_bound)
                    output_image = output_image[:,::-1, :, :] # Convert from BGR to RGB
                    normalized_img = act.patchShow.patchShow_single(output_image, in_range=(-120,120))        
                    normalized_img = np.uint8(normalized_img * 255)
                    new_img = Image.fromarray(normalized_img)
                    #img_file = '{}_{}_{}_{}_{}_img.jpeg'.format(path, type_str, rect_i, str(int(100 * slider_size)), i)
                    #new_img.save(img_file)
                    
                    img.paste(new_img.resize((slider_width, slider_height)), (subrect[0], subrect[1]))
                    img = img.crop(rect)
                    
                    datum_path = '{}_{}_{}_{}_{}.jpeg'.format(path, type_str, rect_i, str(int(100 * slider_size)), i)
                    img.save(datum_path)
                    f.write('{} {}\n'.format(datum_path, str(class_id)))
    '''
                    
            
image_path = imagenet_root + 'ILSVRC2015/Data/CLS-LOC/train/'
annotation_path =  imagenet_root + 'ILSVRC2015/Annotations/CLS-LOC/train/'

if '1k' in type_str:
    synset_names = os.listdir(image_path)

start_time = time.time()

dataset_sum = training_dataset_size + validation_dataset_size + test_dataset_size
all_sum = len(synset_names) * dataset_sum
print all_sum

for synset_index, synset_name in enumerate(synset_names):
    image_names = os.listdir(image_path + synset_name)
    annotation_names = os.listdir(annotation_path + synset_name)
    n1 = [os.path.splitext(n)[0] for n in image_names]
    n2 = [os.path.splitext(n)[0] for n in annotation_names]
    intersection_names = list(set(n1) & set(n2))
    for i in range(dataset_sum):
        if (i + 1) % 100 == 0:
            second = int(time.time() - start_time)
            now_time = time.strftime("%H:%M:%S", time.gmtime(second))
            now_sum = synset_index * dataset_sum + i
            
            estimated = int(float(all_sum) / now_sum * second)
            estimated_time = time.strftime("%H:%M:%S", time.gmtime(estimated))
            estimated_day = estimated / 3600 / 24
            print '[{}/{} {}]Processing synset [{}/{}], image [{}/{}]: {}'.format(now_time, estimated_day, estimated_time, synset_index + 1, len(synset_names), i + 1, dataset_sum, intersection_names[i])
        # Read bounding box.
        bbx_file = open(annotation_path + synset_name + '/' + intersection_names[i] + '.xml')
        xmltree = ET.parse(bbx_file)
        objects = xmltree.findall('object')
        rects = []
        for obj in objects:
            bbx = obj.find('bndbox')
            rects.append([int(it.text) for it in bbx])
            
        img_orig = Image.open(image_path + synset_name + '/' + intersection_names[i] + '.JPEG')
        if img_orig.mode != "RGB":
            img_orig = img_orig.convert("RGB")
            
        if '1k' in type_str:
            class_id = wnid_to_label[synset_name]
        else:
            class_id = original_to_new_class_id[wnid_to_label[synset_name]]
        
        if i < training_dataset_size: # Training dataset. 
            for index, (slider_size, slider_num) in enumerate(dataset):
                generate_datum(img_orig, '{}{}'.format(train_folders[index], intersection_names[i]), \
                               train_files[index], class_id, rects, slider_size, slider_num)
        elif i < training_dataset_size + validation_dataset_size: # Validation dataset
            for index, (slider_size, slider_num) in enumerate(dataset):
                generate_datum(img_orig, '{}{}'.format(val_folders[index], intersection_names[i]), \
                               val_files[index], class_id, rects, slider_size, slider_num)
        else: # Test dataset.
            for index, (slider_size, slider_num) in enumerate(dataset):
                generate_datum(img_orig, '{}{}'.format(test_folders[index], intersection_names[i]), \
                               test_files[index], class_id, rects, slider_size, slider_num)
                               
for f in train_files:
    f.close()
for f in test_files:
    f.close()
            

Overwriting dataset.py


In [15]:
# Sample or for lmdb.

import os
import random
from constant import *
import shutil

#mode = 'nosample'
mode = 'sampl'
is_lmdb = True

func_strs = ['train', 'val', 'test']
type_strs = ['every_grs']
#names = ['0', '10', '20', '30', '40', '50', '60', '70', '80', '90', '100']
names = ['all']
sample_sum = {'train': 343360, 'test': 114060, 'val': 114010}

for func_str in func_strs:
    for type_str in type_strs:
        for name in names:
            #shutil.copyfile('{}dataset/{}_{}_{}.txt'.format(imagenet_root, func_str, type_str, name), \
            #         '{}dataset/{}_{}_{}_unsampled.txt'.format(imagenet_root, func_str, type_str, name))
            with open('{}dataset/{}_{}_{}_unsampled.txt'.format(imagenet_root, func_str, type_str, name)) as f:
                lines = f.readlines()
            if mode == 'nosample':
                sampled = lines
            else:
                sampled = random.sample(lines, sample_sum[func_str])
            if is_lmdb:
                for i in range(len(sampled)):
                    sampled[i] = sampled[i].split('/')[-2] + '/' + sampled[i].split('/')[-1]
            with open('{}dataset/{}_{}_{}.txt'.format(imagenet_root, func_str, type_str, name), 'w') as f:
                f.writelines(sampled)

In [2]:
# Create {}_{}_all.txt.
import os
import random
from constant import *

func_strs = ['train', 'val', 'test']
type_strs = ['crop_img']
names = ['0', '10', '20', '30', '40', '50', '60', '70', '80', '90', '100']
#names = ['0', '20', '40', '60', '80', '100']

#sample_sum = {'train': {'crop': 343360, 'nocrop': 300000}, \
#              'val': {'crop': 114010, 'nocrop': 100000}, \
#              'test': {'crop': 114060, 'nocrop': 100000}}

#sample_sum = {'train': {'aperture': 343360}, \
#              'val': {'aperture': 114010}, \
#              'test': {'aperture': 114060}}

#sample_sum = {'train': {'crop_obj': 343360}, \
#              'val': {'crop_obj': 114010}, \
#              'test': {'crop_obj': 114060}}


#sample_sum = {'train': {'every': 343360}, \
#              'val': {'every': 114010}, \
#              'test': {'every': 114060}}

#sample_sum = {'train': {'crop_img': 343360}, \
#              'val': {'crop_img': 114010}, \
#              'test': {'crop_img': 114060}}


sample_sum = {'train': {'every_grs': 343360}, \
              'val': {'every_grs': 114010}, \
              'test': {'every_grs': 114060}}

for func_str in func_strs:
    for type_str in type_strs:
        lines = []
        for name in names:
            f = open('{}dataset/{}_{}_{}.txt'.format(imagenet_root, func_str, type_str, name), 'r')
            ls = f.readlines()
            f.close()
            lines = lines + ls
        f = open('{}dataset/{}_{}_all_unsampled.txt'.format(imagenet_root, func_str, type_str), 'w')
        f.writelines(lines)
        f.close()
        
        
        sampled = random.sample(lines, sample_sum[func_str][type_str])
        f = open('{}dataset/{}_{}_all.txt'.format(imagenet_root, func_str, type_str), 'w')
        f.writelines(sampled)
        f.close()

In [3]:
# Sanity check.
import os
from constant import *

func_strs = ['train', 'val', 'test']
type_strs = ['crop_img']
names = ['0', '10', '20', '30', '40', '50', '60', '70', '80', '90', '100', 'all']

for func_str in func_strs:
    for type_str in type_strs:
        for name in names:
            filename = '{}dataset/{}_{}_{}.txt'.format(imagenet_root, func_str, type_str, name) 
            f = open(filename)
            lines = f.readlines()
            f.close()
            print '{:10d} {}'.format(len(lines), filename)

     34336 /data2/haow3/data/imagenet/dataset/train_crop_img_0.txt
     34336 /data2/haow3/data/imagenet/dataset/train_crop_img_10.txt
     34336 /data2/haow3/data/imagenet/dataset/train_crop_img_20.txt
     34336 /data2/haow3/data/imagenet/dataset/train_crop_img_30.txt
     34336 /data2/haow3/data/imagenet/dataset/train_crop_img_40.txt
     34336 /data2/haow3/data/imagenet/dataset/train_crop_img_50.txt
     34336 /data2/haow3/data/imagenet/dataset/train_crop_img_60.txt
     34336 /data2/haow3/data/imagenet/dataset/train_crop_img_70.txt
     34336 /data2/haow3/data/imagenet/dataset/train_crop_img_80.txt
     34336 /data2/haow3/data/imagenet/dataset/train_crop_img_90.txt
     34336 /data2/haow3/data/imagenet/dataset/train_crop_img_100.txt
    343360 /data2/haow3/data/imagenet/dataset/train_crop_img_all.txt
     11401 /data2/haow3/data/imagenet/dataset/val_crop_img_0.txt
     11401 /data2/haow3/data/imagenet/dataset/val_crop_img_10.txt
     11401 /data2/haow3/data/imagenet/dataset/val_cr

In [ ]:
#%%file transparent.py
# Make white color transparent and crop bounding box. For nocrop_obj/crop_obj.
from constant import *
from PIL import Image, ImageDraw
from matplotlib.pyplot import imshow
import numpy as np
import os

image_names = os.listdir(shapenet_root + 'object_orig/')
print len(image_names)

ratio = []

for i, image_name in enumerate(image_names):
    print '[{}/{}]: {}'.format(i, len(image_names), image_name)
    img = Image.open(shapenet_root + 'object_orig/' + image_name)
    
    img = img.convert("RGBA")
    pixdata = img.load()
    
    scanline_x = np.zeros(img.size[0])
    scanline_y = np.zeros(img.size[1])
    
    for x in xrange(img.size[0]):
        for y in xrange(img.size[1]):
            if pixdata[x, y] == (255, 255, 255, 255):
                pixdata[x, y] = (255, 255, 255, 0)
            else:
                scanline_x[x] = 1
                scanline_y[y] = 1
                
    # Get bounding box.
    rect = [0, 0, 0, 0]
    for i in range(len(scanline_x)):
        if scanline_x[i] != 0:
            rect[0] = i
            break
    for i in range(len(scanline_x)):
        if scanline_x[len(scanline_x) - i - 1] != 0:
            rect[2] = len(scanline_x) - i - 1
            break
    for i in range(len(scanline_y)):
        if scanline_y[i] != 0:
            rect[1] = i
            break
    for i in range(len(scanline_y)):
        if scanline_y[len(scanline_y) - i - 1] != 0:
            rect[3] = len(scanline_y) - i - 1
            break
            
    ratio.append((rect[2] - rect[0]) / (rect[3] - rect[1]))
            
    img = img.crop(rect)
            
    img.save(shapenet_root + 'object_crop/' + image_name, 'PNG')


In [ ]:
import os
from constant import *
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

f = open(caffe_root + 'data/ilsvrc12/synset_words.txt')
label_to_wnid = f.readlines()
wnid_to_label = {}
for label in range(len(label_to_wnid)):
    wnid = label_to_wnid[label].split(' ')[0]
    wnid_to_label[wnid] = label

folders = os.listdir(imagenet_root + 'ILSVRC2015/Data/CLS-LOC/train/')
count = []
for folder in folders:
    image_names = os.listdir(imagenet_root + 'ILSVRC2015/Data/CLS-LOC/train/' + folder)
    annotation_names = os.listdir(imagenet_root + 'ILSVRC2015/Annotations/CLS-LOC/train/' + folder)
    n1 = [os.path.splitext(n)[0] for n in image_names]
    n2 = [os.path.splitext(n)[0] for n in annotation_names]
    intersection_names = list(set(n1) & set(n2))
    count.append(len(intersection_names))

x = [i for i in range(len(count))]
count = sorted(count)
plt.plot(x, count)

In [ ]:
# Randomly sample classes for experiment.
import os
from constant import *
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import random


imagenet_labels_filename = imagenet_root + 'ilsvrc12/synset_words.txt'
label_to_wnid = np.loadtxt(imagenet_labels_filename, str, delimiter='\t')

synset_name_to_original_id = {}
for original_id in range(len(label_to_wnid)):
    synset_name = label_to_wnid[original_id].split(' ')[0]
    synset_name_to_original_id[synset_name] = original_id

image_path = imagenet_root + 'ILSVRC2015/Data/CLS-LOC/train/'
annotation_path =  imagenet_root + 'ILSVRC2015/Annotations/CLS-LOC/train/'

count = []
sampled = []
for synset_index, synset_name in enumerate(os.listdir(image_path)):
    image_names = os.listdir(image_path + synset_name)
    annotation_names = os.listdir(annotation_path + synset_name)
    n1 = [os.path.splitext(n)[0] for n in image_names]
    n2 = [os.path.splitext(n)[0] for n in annotation_names]
    intersection_names = list(set(n1) & set(n2))
    count.append(len(intersection_names))
    if len(intersection_names) >= 500:
        sampled.append(synset_name)

count = sorted(count)
plt.plot(count)

sampled = random.sample(sampled, 100)
print sampled

new_to_original_class_id = []
original_to_new_class_id = {}
for synset_name in sampled:
    new_to_original_class_id.append(synset_name_to_original_id[synset_name])
new_to_original_class_id = sorted(new_to_original_class_id)
print new_to_original_class_id

for i, original_id in enumerate(new_to_original_class_id):
    original_to_new_class_id[original_id] = i
print original_to_new_class_id

In [ ]:
# Generate label list.
import os
from constant import *

imagenet_labels_filename = imagenet_root + 'ilsvrc12/synset_words.txt'
lines = open(imagenet_labels_filename).readlines()

original_id_to_name = []
for line in lines:
    wnid = line.split(' ')[0]
    name = line[(len(wnid) + 1):-1]
    original_id_to_name.append(name)
    
    
label = [None for i in range(len(synset_names))]

for new_id, original_id in enumerate(new_to_original_class_id):
    label[new_id] = original_id_to_name[original_id]
    print label[new_id]
    
    

In [1]:
# Synthesize occluders.
import os
from constant import *
import caffe
import numpy as np
caffe.set_mode_gpu() # uncomment this if gpu processing is available
import act_max as act
import time

# Parameters.
act_args = lambda: None
act_args.xy = 0
act_args.n_iters = 200
act_args.L2 = 0.99
act_args.start_lr = 8.0
act_args.end_lr = 1e-10
act_args.seed = 0
act_args.opt_layer = "fc6"
act_args.act_layer = "fc8_occlusion"
act_args.init_file = "None"
act_args.clip = 0
act_args.bound = synthesizing_root + "act_range/3x/fc6.txt"
act_args.debug = 0
act_args.output_dir = synthesizing_root + "output/baseline"

#act_args.net_weights = result_root + "model/finetune_alexnet_nocrop_0/finetune_alexnet_nocrop_0.caffemodel"
#act_args.net_definition = result_root + "model/finetune_alexnet_nocrop_0/deploy.prototxt"
act_args.net_weights = result_root + "model/finetune_alexnet_shapenet_all/finetune_alexnet_shapenet_all.caffemodel"
act_args.net_definition = result_root + "model/finetune_alexnet_shapenet_all/deploy.prototxt"
act_args.generator_weights = synthesizing_root + "nets/upconv/fc6/generator.caffemodel"
act_args.generator_definition = synthesizing_root + "nets/upconv/fc6/generator.prototxt"

params = [
    {
        'layer': act_args.act_layer,
        'iter_n': act_args.n_iters,
        'L2': act_args.L2,
        'start_step_size': act_args.start_lr,
        'end_step_size': act_args.end_lr
    }
]

# Networks.
generator = caffe.Net(act_args.generator_definition, act_args.generator_weights, caffe.TEST)
net = caffe.Classifier(act_args.net_definition, act_args.net_weights,
        mean = np.float32([104.0, 117.0, 123.0]), # ImageNet mean
        channel_swap = (2,1,0)) # the reference model has channels in BGR order instead of RGB

# input / output layers in generator
gen_in_layer = "feat"
gen_out_layer = "deconv0"

# shape of the code being optimized
shape = generator.blobs[gen_in_layer].data.shape

# Fix the seed
np.random.seed(act_args.seed)

if act_args.init_file != "None":
    start_code, start_image = act.get_code(act_args.init_file, act_args.opt_layer)
    print "Loaded start code: ", start_code.shape
else:
    start_code = np.random.normal(0, 1, shape)

# Load the activation range
upper_bound = lower_bound = None

# Set up clipping bounds
if act_args.bound != "":
    n_units = shape[1]
    upper_bound = np.loadtxt(act_args.bound, delimiter=' ', usecols=np.arange(0, n_units), unpack=True)
    upper_bound = upper_bound.reshape(start_code.shape)

    # Lower bound of 0 due to ReLU
    lower_bound = np.zeros(start_code.shape)

#class_sum = 100
#num_sum = 5

class_sum = 29
num_sum = 5

start_time = time.time()
for class_id in range(class_sum):
    act_args.unit = class_id
    for num in range(num_sum):
        start_code = np.random.normal(0, 1, shape)
        # Optimize a code via gradient ascent
        output_image = act.activation_maximization(net, generator, gen_in_layer, gen_out_layer, start_code, params, 
                clip=act_args.clip, unit=act_args.unit, xy=act_args.xy, debug=act_args.debug,
                upper_bound=upper_bound, lower_bound=lower_bound)
        # Save image
        act.save_image(output_image, '{}synthesized/{}_{}.jpg'.format(shapenet_root, class_id, num))
        
        if num % 5 == 0:
            second = int(time.time() - start_time)
            now_time = time.strftime("%H:%M:%S", time.gmtime(second))

            estimated = int(float(class_sum * num_sum) / (class_id * num_sum + num + 1) * second)
            estimated_time = time.strftime("%H:%M:%S", time.gmtime(estimated))
            estimated_day = estimated / 3600 / 24
            print '[{}/{} {}] class: {}, num: {}'.format(now_time, estimated_day, estimated_time, class_id, num + 1)

        
if act_args.debug:
    act.save_image(output_image, "./debug/%s.jpg" % str(act_args.n_iters).zfill(3))
    
print 'done'


[00:00:04/0 00:09:40] class: 0, num: 1
[00:00:23/0 00:09:15] class: 1, num: 1
[00:00:43/0 00:09:26] class: 2, num: 1
[00:01:03/0 00:09:30] class: 3, num: 1
[00:01:22/0 00:09:26] class: 4, num: 1
[00:01:42/0 00:09:28] class: 5, num: 1
[00:02:02/0 00:09:30] class: 6, num: 1
[00:02:21/0 00:09:27] class: 7, num: 1
[00:02:42/0 00:09:32] class: 8, num: 1
[00:03:05/0 00:09:43] class: 9, num: 1
[00:03:27/0 00:09:48] class: 10, num: 1
[00:03:50/0 00:09:55] class: 11, num: 1
[00:04:12/0 00:09:59] class: 12, num: 1
[00:04:34/0 00:10:01] class: 13, num: 1
[00:04:56/0 00:10:04] class: 14, num: 1
[00:05:19/0 00:10:08] class: 15, num: 1
[00:05:41/0 00:10:10] class: 16, num: 1
[00:06:03/0 00:10:12] class: 17, num: 1
[00:06:26/0 00:10:15] class: 18, num: 1
[00:06:48/0 00:10:16] class: 19, num: 1
[00:07:10/0 00:10:17] class: 20, num: 1
[00:07:32/0 00:10:18] class: 21, num: 1
[00:07:52/0 00:10:16] class: 22, num: 1
[00:08:11/0 00:10:13] class: 23, num: 1
[00:08:31/0 00:10:12] class: 24, num: 1
[00:08:51/

In [7]:
# Calculate mean image.
import numpy as np
from PIL import Image, ImageDraw, ImageColor
from constant import *
import matplotlib.pyplot as plt
%matplotlib inline
import random

img_images = []
for class_id in range(100):
    filename = '{}_{}.jpg'.format(class_id, random.randint(0, 4))
    img = Image.open(imagenet_root + 'synthesized/' + filename)
    img_images.append(img)

avg = np.array(img_images[0], dtype = np.float32)
for i in range(1, 100):
    avg = avg + np.array(img_images[i])
avg /= 100
m = Image.fromarray(np.uint8(avg))
print m
m.save(imagenet_root + 'synthesized/mean.jpg')

<PIL.Image.Image image mode=RGB size=227x227 at 0x2B1B5002CA50>
